## 合并

### 添加

In [8]:
# 添加行（append）
df = pd.DataFrame([[1, 2], [3, 4]], columns=list('AB'))
df

# 利用序列添加行（必须指定name）
s = pd.Series({'A':10,'B':11},name='new_row')
df.append(s)

#添加表
df2 = pd.DataFrame([[5, 6], [7, 8]], columns=list('AB'))
df.append(df2)


# 添加列（assign）
# 列名直接由参数指定
df.assign(col1=df2['C'], col2=df2["D"])

,A,B
0,1,2
1,3,4


,A,B
0,1,2
1,3,4
new_row,10,11


,A,B
0,1,2
1,3,4
0,5,6
1,7,8


### 填充

combine、update

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.combine.html

In [23]:
# combine方法是按照表的顺序轮流进行逐列循环的，而且自动索引对齐，缺失值为NaN
df1 = pd.DataFrame({'A': [5, 0], 'B': [2, 4]})
df2 = pd.DataFrame({'A': [1, 1], 'B': [3, 3]})
df1.combine(df2, np.minimum)

df2 = pd.DataFrame({'B': [8, 7], 'C': [6, 5]},index=[1,2])
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y)

# combine_first,用df2填补df1的缺失值，功能比较简单，但很多时候会比combine更常用
df1 = pd.DataFrame({'A': [None, 0], 'B': [None, 4]})
df2 = pd.DataFrame({'A': [1, 1], 'B': [3, 3]})
df1.combine_first(df2)

,A,B
0,1,2
1,0,3


,A,B,C
0,NaN,NaN,NaN
1,NaN,8.0,6.0
2,NaN,7.0,5.0


,A,B
0,1.0,3.0
1,0.0,4.0


In [26]:
# update
# ①返回的框索引只会与被调用框的一致（默认使用左连接，下一节会介绍）
# ②第二个框中的nan元素不会起作用
# ③没有返回值，直接在df上操作

df1 = pd.DataFrame({'A': [1, 2, 3],
                    'B': [400, 500, 600]})
df2 = pd.DataFrame({'B': [4, 5, 6],
                    'C': [7, 8, 9]})
df1.update(df2)
df1


df1 = pd.DataFrame({'A': ['a', 'b', 'c'],
                    'B': ['x', 'y', 'z']})
df2 = pd.DataFrame({'B': ['d', 'e']}, index=[1,2])
df1.update(df2)
df1

,A,B
0,1,4
1,2,5
2,3,6


,A,B
0,a,x
1,b,d
2,c,e


### 合并

merge、jion、Concat

https://nbviewer.jupyter.org/github/GYHHAHA/Joyful-Pandas/blob/master/%E7%AC%AC5%E7%AB%A0%20%E5%90%88%E5%B9%B6.ipynb#%E4%B8%80%E3%80%81append%E4%B8%8Eassign


https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#database-style-dataframe-or-named-series-joining-merging


https://www.bookstack.cn/read/PandasCookbook/spilt.4.9.md


1. merge以列为连接点，jion以index为连接点。通过设置函数里的参数，可以使两个功能相同
https://zhuanlan.zhihu.com/p/45442554

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

#### merge

merge函数的作用是将两个pandas对象横向合并，遇到重复的索引项时会使用笛卡尔积，默认inner连接，可选left、outer、right连接

In [28]:

left = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1'],
                      'A': ['A0', 'A1', 'A2', 'A3'],
                      'B': ['B0', 'B1', 'B2', 'B3']}) 
right = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})
right2 = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'C': ['C0', 'C1', 'C2', 'C3']})

# 以key1为准则连接，如果具有相同的列，则默认suffixes=('_x','_y')
pd.merge(left, right, on='key1')


# 默认使用inner连接，因为merge只能横向拼接，所以取行向上keys的交集，下面看如果使用how=outer参数
# 注意：这里的how就是concat的join
pd.merge(left, right, how='outer', on=['key1','key2'])

,key1,key2_x,A,B,key2_y,C,D
0,K0,K0,A0,B0,K0,C0,D0
1,K0,K1,A1,B1,K0,C0,D0
2,K1,K0,A2,B2,K0,C1,D1
3,K1,K0,A2,B2,K0,C2,D2
4,K2,K1,A3,B3,K0,C3,D3


,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K1,A3,B3,NaN,NaN
5,K2,K0,NaN,NaN,C3,D3


#### jion

join函数作用是将多个pandas对象横向拼接，遇到重复的索引项时会使用笛卡尔积，默认左连接，可选inner、outer、right连接

对于many_to_one模式下的合并，往往join更为方便

In [30]:


left = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3'],
                     'key': ['K0', 'K1', 'K0', 'K1']})
right = pd.DataFrame({'C': ['C0', 'C1'],
                      'D': ['D0', 'D1']},
                     index=['K0', 'K1'])
left.join(right, on='key')

,A,B,key,C,D
0,A0,B0,K0,C0,D0
1,A1,B1,K1,C1,D1
2,A2,B2,K0,C0,D0
3,A3,B3,K1,C1,D1


#### concat

concat方法可以在两个维度上拼接，默认纵向凭借（axis=0），拼接方式默认外连接

In [31]:
df1 = pd.DataFrame({'A': ['A0', 'A1'],
                    'B': ['B0', 'B1']},
                    index = [0,1])
df2 = pd.DataFrame({'A': ['A2', 'A3'],
                    'B': ['B2', 'B3']},
                    index = [2,3])
df3 = pd.DataFrame({'A': ['A1', 'A3'],
                    'D': ['D1', 'D3'],
                    'E': ['E1', 'E3']},
                    index = [1,3])

pd.concat([df1,df2])

# axis=1时沿列方向拼接
pd.concat([df1,df2],axis=1)

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


,A,B,A,B
0,A0,B0,NaN,NaN
1,A1,B1,NaN,NaN
2,NaN,NaN,A2,B2
3,NaN,NaN,A3,B3
